<a href="https://colab.research.google.com/github/nevemarpole/DissertationProject/blob/main/Generative_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install necessary libraries
!pip install transformers

In [ ]:
#Imports
import torch
import tensorflow as tf
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import random
from math import log10
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
import pickle

In [ ]:
#Fetch name of GPU in use
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
#Downloads
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

#Load in saved GPT-2 model
gpt2_model = pickle.load(open('drive/MyDrive/Colab Notebooks/Dissertation/GPT-2/finetuned_GPT2_model.pkl', 'rb'))
gpt2_model.to(device)

In [ ]:
#Tokenizes, converts all text to lower case, 
#removes stop words, stems the text passed to it
def rework_text(text, token, stem):

  #Tokenize the text to make it a list of lists
  #Allowing words to be accessed indivdually
  if token == True:
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    changed_text = []
    for string in text:
        changed_text.append(tokenizer.tokenize(string))
        
    text = changed_text

    print("Text tokenized")



  if stem == True:
    #Remove stop words and make all charcters lower case
    documents = []
    english_stopwords = stopwords.words('english')
    for tok_document in text:
        documents.append([word.lower() for word in tok_document if word not in english_stopwords])
    
    #Stem all words
    stemmed_documents = []
    for part in documents:
        stemmed_documents.append([sb_stemmer.stem(word) for word in part]) 
    changed_text = stemmed_documents

    print("Stop words removed and stemmed")

  return changed_text

In [ ]:
#Say hello to the user
print('Hi there!')


#Until the user asks to stop the chatbot will talk to them
stop = False

while not stop:

  found = False
  text = []
  
  #Ask the user for an input
  user_input = input('Please enter a sentence, or to bring this conversation to an end ask me to stop:\n')
  

  #Lower and tokenize the user's input to make it workable
  user_input = user_input.lower()
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  input_tokenized = []
  input_tokenized.append(tokenizer.tokenize(user_input))
  
  #As input_tokenized makes a list of lists use this to just get
  #the list of words as tokens
  token_list = input_tokenized[0]

  #If the input is stop then end the conversation
  if token_list[0] == "stop" and len(token_list) == 1:
      print("Goodbye!")
      stop = True


  #The input isn't stop so a response must be chosen        
  if stop == False: 
      #Set to the correct tokenizer
      tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
      
      #Make sure model is in evaluation mode
      gpt2_model.eval()

      output = []

      #enocde the user input for the model to understand
      encoded = torch.tensor(tokenizer.encode(user_input)).unsqueeze(0)
      encoded = generated.to(device)

      #Sets parameters for model generation
      #Here the generation is done based on the user's encoded input
      #it will look forward 100 places when search for words to use to construct
      #it will be capped at a length of 40, and only return 1 generated sequence
      sample_outputs = gpt2_model.generate(
                                      encoded, 
                                      do_sample=True,   
                                      top_k=100, 
                                      max_length = 40,
                                      top_p=0.95, 
                                      num_return_sequences=1
                                      )
      
      #Decode the output from numbers to string
      #And remove the user input which is at the beginning
      output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
      output = output.replace(user_input,'')
      print(output, "\n")